In [8]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\eriveraj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os, random
from dotenv import load_dotenv

In [2]:
BASE_DIR = r"C:\AI_Agents_Aire"
DATA_DIR = os.path.join(BASE_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)

In [3]:
from dotenv import load_dotenv
import os

# Ruta donde está tu .env (ajusta si es distinta)
BASE_DIR = r"C:\AI_Agents_Aire"  # por ejemplo
dotenv_path = os.path.join(BASE_DIR, ".env")

print("Usando .env en:", dotenv_path)
load_dotenv(dotenv_path=dotenv_path)

API_KEY = os.getenv("OPENAQ_API_KEY")
print("Longitud de la API_KEY:", len(API_KEY) if API_KEY else None)

if not API_KEY:
    raise ValueError("No se encontró OPENAQ_API_KEY en el .env")

url = "https://api.openaq.org/v3/parameters"
headers = {"X-API-Key": API_KEY}

r = requests.get(url, headers=headers)
print("Status:", r.status_code)
print("Respuesta JSON:", r.json())


Usando .env en: C:\AI_Agents_Aire\.env
Longitud de la API_KEY: 64
Status: 200
Respuesta JSON: {'meta': {'name': 'openaq-api', 'website': '/', 'page': 1, 'limit': 100, 'found': 46}, 'results': [{'id': 1, 'name': 'pm10', 'units': 'µg/m³', 'displayName': 'PM10', 'description': 'Particulate matter less than 10 micrometers in diameter mass concentration'}, {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5', 'description': 'Particulate matter less than 2.5 micrometers in diameter mass concentration'}, {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': 'O₃ mass', 'description': 'Ozone mass concentration'}, {'id': 4, 'name': 'co', 'units': 'µg/m³', 'displayName': 'CO mass', 'description': 'Carbon Monoxide mass concentration'}, {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'displayName': 'NO₂ mass', 'description': 'Nitrogen Dioxide mass concentration'}, {'id': 6, 'name': 'so2', 'units': 'µg/m³', 'displayName': 'SO₂ mass', 'description': 'Sulfur Dioxide mass concentration'}, {

Función para elegir el rango de fechas (modo histórico / real-time)

In [4]:
def get_time_window(mode="historico"):
    """
    Devuelve START y END según el modo:
    - 'historico': rango fijo donde sabemos que OpenAQ tiene datos.
    - 'tiempo_real': últimas 24 horas desde ahora.
    """
    if mode == "historico":
        # Ejemplo: semana del 13 al 20 de marzo de 2016
        end = datetime(2016, 3, 20)
        start = end - timedelta(days=7)
    elif mode == "tiempo_real":
        end = datetime.utcnow()
        start = end - timedelta(hours=24)
    else:
        raise ValueError("Modo debe ser 'historico' o 'tiempo_real'")
    return start, end


Descargar datos desde OpenAQ (función con los dos modos)

In [7]:
from dotenv import load_dotenv, find_dotenv

# ==== CONFIGURACIÓN GENERAL ====

# Cargar .env y leer la API key real
print("Usando .env en:", find_dotenv())
load_dotenv()

API_KEY = os.getenv("OPENAQ_API_KEY")
if not API_KEY:
    raise ValueError("Falta OPENAQ_API_KEY en el .env")

# Header correcto que usará TODA la función
headers = {"X-API-Key": API_KEY}

BASE_URL = "https://api.openaq.org/v3"

LIMA_COORDS = "-12.0464,-77.0428"
RADIUS = 20000  # 20 km a la redonda
PARAMS = ["pm10"]  # para Lima, pm10 es el más estable


# ==== FUNCIÓN PRINCIPAL ====

def descargar_openaq(mode="historico"):
    start, end = get_time_window(mode)
    print(f"Modo: {mode} | Desde {start} hasta {end}")

    # 1) Encontrar id de pm10
    resp = requests.get(
        f"{BASE_URL}/parameters",
        headers=headers,
        params={"limit": 100},
        timeout=20,
    )
    resp.raise_for_status()
    params_data = resp.json()["results"]

    name_to_id = {p["name"]: p["id"] for p in params_data if p["name"] in PARAMS}
    print("Mapa parámetro → id:", name_to_id)

    rows = []

    for pname in PARAMS:
        param_id = name_to_id[pname]
        print(f"\n=== Parámetro {pname} (id={param_id}) ===")

        # 2) Locaciones cerca de Lima que midan ese parámetro
        loc_resp = requests.get(
            f"{BASE_URL}/locations",
            headers=headers,
            params={
                "iso": "PE",
                "coordinates": LIMA_COORDS,
                "radius": RADIUS,
                "parameters_id": param_id,
                "limit": 100,
            },
            timeout=20,
        )
        loc_resp.raise_for_status()
        locations = loc_resp.json().get("results", [])
        print(f"Locations encontradas: {len(locations)}")

        for loc in locations:
            loc_id = loc["id"]
            loc_name = loc.get("name")
            loc_coords = loc.get("coordinates") or {}
            loc_lat = loc_coords.get("latitude")
            loc_lon = loc_coords.get("longitude")

            print(f"  Location {loc_id} ({loc_name})")

            # 3) Sensores de la localización
            sens_resp = requests.get(
                f"{BASE_URL}/locations/{loc_id}/sensors",
                headers=headers,
                params={"limit": 100},
                timeout=20,
            )
            sens_resp.raise_for_status()
            sens_data = sens_resp.json().get("results", [])

            sensors = [
                s for s in sens_data
                if s.get("parameter", {}).get("name") == pname
            ]
            if not sensors:
                continue

            # Para no tardar mucho, usamos solo el primer sensor de cada location
            sensor = sensors[0]
            sensor_id = sensor["id"]
            print(f"    Usando sensor {sensor_id}")

            # 4) Medidas horarias en el rango elegido
            page = 1
            while True:
                meas_resp = requests.get(
                    f"{BASE_URL}/sensors/{sensor_id}/hours",
                    headers=headers,
                    params={
                        "date_from": start.isoformat(timespec="seconds") + "Z",
                        "date_to":   end.isoformat(timespec="seconds")   + "Z",
                        "limit": 200,
                        "page": page,
                    },
                    timeout=20,
                )

                if meas_resp.status_code in (408, 504, 422):
                    print(f"      Página {page}: status {meas_resp.status_code}, corto aquí.")
                    break

                meas_resp.raise_for_status()
                meas_data = meas_resp.json()
                results = meas_data.get("results", [])
                print(f"      Página {page}: {len(results)} registros")

                if not results:
                    break

                for m in results:
                    period = m.get("period") or {}
                    dt_from = period.get("datetimeFrom") or {}
                    timestamp_utc = dt_from.get("utc")

                    param_info = m.get("parameter") or {}

                    rows.append({
                        "timestamp_utc": timestamp_utc,
                        "location_id": loc_id,
                        "location_name": loc_name,
                        "sensor_id": sensor_id,
                        "parameter": pname,
                        "value": m.get("value"),
                        "unit": param_info.get("units"),
                        "lat": loc_lat,
                        "lon": loc_lon,
                    })

                page += 1

    df = pd.DataFrame(rows)
    print("\nMediciones descargadas:", len(df))
    return df


Usando .env en: c:\AI_Agents_Aire\.env


In [ ]:
# Modo estable (demo): histórico
df_hist = descargar_openaq(mode="historico")

Modo: historico | Desde 2016-03-13 00:00:00 hasta 2016-03-20 00:00:00
Mapa parámetro → id: {'pm10': 1}

=== Parámetro pm10 (id=1) ===
Locations encontradas: 15
  Location 819 (Ate)
    Usando sensor 1448
      Página 1: 200 registros
      Página 2: 200 registros
      Página 3: 200 registros
      Página 4: 200 registros
      Página 5: 200 registros
      Página 6: 200 registros
      Página 7: 200 registros
      Página 8: 200 registros
      Página 9: 200 registros
      Página 10: 200 registros
      Página 11: 200 registros
      Página 12: 200 registros
      Página 13: 200 registros
      Página 14: 200 registros
      Página 15: 200 registros
      Página 16: 200 registros
      Página 17: 200 registros
      Página 18: 200 registros
      Página 19: 200 registros
      Página 20: 200 registros
      Página 21: 200 registros
      Página 22: 200 registros
      Página 23: 200 registros
      Página 24: 200 registros
      Página 25: 200 registros
      Página 26: 200 registros

In [9]:
# Modo “monitoreo en vivo”: últimas 24h
df_live = descargar_openaq(mode="tiempo_real")

Modo: tiempo_real | Desde 2025-11-07 06:48:27.765533 hasta 2025-11-08 06:48:27.765533
Mapa parámetro → id: {'pm10': 1}

=== Parámetro pm10 (id=1) ===
Locations encontradas: 15
  Location 819 (Ate)
    Usando sensor 1448
      Página 1: 200 registros
      Página 2: 200 registros
      Página 3: 200 registros
      Página 4: 200 registros
      Página 5: 200 registros
      Página 6: 200 registros
      Página 7: 200 registros
      Página 8: 200 registros
      Página 9: 200 registros
      Página 10: 200 registros
      Página 11: 200 registros
      Página 12: 200 registros
      Página 13: 200 registros
      Página 14: 200 registros
      Página 15: 200 registros
      Página 16: 200 registros
      Página 17: 200 registros
      Página 18: 200 registros
      Página 19: 200 registros
      Página 20: 200 registros
      Página 21: 200 registros
      Página 22: 200 registros
      Página 23: 200 registros
      Página 24: 200 registros
      Página 25: 200 registros
      Página 2

In [32]:
print("Header que envío:", headers)

Header que envío: {'X-API-Key': 'OPENAQ_API_KEY'}


Construir el dataset con viento, humedad y hora

In [11]:
import random
import numpy as np

DATA_DIR = os.path.join(BASE_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)

def construir_dataset(df, nombre_archivo):
    # 1) Pivot: una fila por timestamp y distrito
    pivot = df.pivot_table(
        index=["timestamp_utc", "location_name"],
        columns="parameter",
        values="value",
        aggfunc="mean"
    ).reset_index()

    # 2) Añadir lat/lon por distrito
    coords = df[["location_name", "lat", "lon"]].drop_duplicates()
    pivot = pivot.merge(coords, on="location_name", how="left")

    # 3) Renombrar columnas
    pivot.rename(
        columns={
            "timestamp_utc": "timestamp",
            "location_name": "district"
        },
        inplace=True
    )

    # 4) Asegurar columnas de parámetros
    for col in ["pm25", "pm10", "no2", "o3"]:
        if col not in pivot.columns:
            pivot[col] = np.nan

    # 5) Clima simulado
    random.seed(42)
    pivot["timestamp"] = pd.to_datetime(pivot["timestamp"])

    noise_temp = pd.Series(
        [random.uniform(-1, 1) for _ in range(len(pivot))],
        index=pivot.index
    )
    noise_hum = pd.Series(
        [random.uniform(-10, 10) for _ in range(len(pivot))],
        index=pivot.index
    )
    noise_wind = pd.Series(
        [random.uniform(-1, 1) for _ in range(len(pivot))],
        index=pivot.index
    )

    pivot["temp_c"] = 18 + (pivot["timestamp"].dt.hour - 12) * 0.3 + noise_temp
    pivot["humidity_pct"] = 70 + noise_hum
    pivot["wind_ms"] = 2 + noise_wind

    # 6) Orden de columnas
    cols = [
        "timestamp", "district", "pm25", "pm10", "no2", "o3",
        "temp_c", "humidity_pct", "wind_ms", "lat", "lon"
    ]
    pivot = pivot[cols]

    out_path = os.path.join(DATA_DIR, nombre_archivo)
    pivot.to_csv(out_path, index=False)
    print(f"Guardado dataset en: {out_path}")
    return pivot


In [12]:
historial = construir_dataset(df_hist, "historical_readings.csv")
historial.head()


Guardado dataset en: C:\AI_Agents_Aire\data\historical_readings.csv


timestamp  district  pm25   pm10  no2  o3     temp_c  \
0 2016-03-12 04:00:00+00:00  HUACHIPA   NaN  109.0  NaN NaN  15.878854   
1 2016-03-12 05:00:00+00:00  HUACHIPA   NaN  113.0  NaN NaN  14.950022   
2 2016-03-12 06:00:00+00:00  HUACHIPA   NaN  101.0  NaN NaN  15.750059   
3 2016-03-12 07:00:00+00:00  HUACHIPA   NaN   98.1  NaN NaN  15.946421   
4 2016-03-12 08:00:00+00:00  HUACHIPA   NaN  131.0  NaN NaN  17.272942   

   humidity_pct   wind_ms        lat        lon  
0     76.603072  2.390308 -12.017094 -76.948597  
1     60.357718  2.327283 -12.017094 -76.948597  
2     76.879161  1.113699 -12.017094 -76.948597  
3     75.410237  2.348253 -12.017094 -76.948597  
4     78.593325  1.421261 -12.017094 -76.948597